<a href="https://colab.research.google.com/github/carolynw898/STAT946Proj/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from utils import lossFunc, relativeErr
import torch
from torch.utils.data import DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
n_embd = 512
timesteps = 1000
batch_size = 1
learning_rate = 1e-4
num_epochs = 10
blockSize = 32
testBlockSize = 400
numVars = 1
numYs = 1
numPoints = 250
target = 'Skeleton'
const_range = [-2.1, 2.1]
trainRange = [-3.0, 3.0]
decimals = 8
addVars = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
import numpy as np
import glob
from utils import processDataFiles, CharDataset
import random
import json

files = glob.glob("/content/drive/MyDrive/Colab Notebooks/STAT946_proj/data/1_var_train.json")
text = processDataFiles(files)
text = text.split('\n') # convert the raw text to a set of examples
# skeletons = []
skeletons = [json.loads(item)['Skeleton'] for item in text if item.strip()]
all_tokens = set()
for eq in skeletons:
    all_tokens.update(tokenize_equation(eq))
integers = {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}
all_tokens.update(integers)  # add all integers to the token set
tokens = sorted(list(all_tokens) + ['_', 'T', '<', '>', ':'])  # special tokens
trainText = text[:-1] if len(text[-1]) == 0 else text
random.shuffle(trainText) # shuffle the dataset, it's important specailly for the combined number of variables experiment
train_dataset = CharDataset(trainText, blockSize, tokens=tokens, numVars=numVars,
                        numYs=numYs, numPoints=numPoints, target=target, addVars=addVars,
                        const_range=const_range, xRange=trainRange, decimals=decimals)

idx = np.random.randint(train_dataset.__len__())
inputs, outputs, points, variables = train_dataset.__getitem__(idx)
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\noutputs:{}\nvariables:{}'.format(idx,outputs,variables))

data has 498795 examples, 27 unique.
id:389634
outputs:C*x1/(C*x1+C)+C>__________________
variables:1


In [20]:
files = glob.glob("/content/drive/MyDrive/Colab Notebooks/STAT946_proj/data/1_var_test.json")
textTest = processDataFiles([files[0]])
textTest = textTest.split('\n') # convert the raw text to a set of examples
test_dataset = CharDataset(textTest, blockSize, tokens=tokens, numVars=numVars,
                        numYs=numYs, numPoints=numPoints, target=target, addVars=addVars,
                        const_range=const_range, xRange=trainRange, decimals=decimals)

# print a random sample
idx = np.random.randint(test_dataset.__len__())
inputs, outputs, points, variables = test_dataset.__getitem__(idx)
print(points.min(), points.max())
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\noutputs:{}\nvariables:{}'.format(idx,outputs,variables))

test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        pin_memory=True,
        shuffle=False,
        num_workers=2
    )

data has 967 examples, 27 unique.
tensor(-25.4849) tensor(2.6692)
id:719
outputs:C*sin(C*x1**5+C*x1**4+C*x1**3+C*x1**2+C*x1)
variables:1


In [42]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from typing import Tuple
from utils import CharDataset
from sympy import sympify, SympifyError
from tqdm import tqdm


# from SymbolicGPT: https://github.com/mojivalipour/symbolicgpt/blob/master/models.py
class PointNetConfig:
    """base PointNet config"""

    def __init__(
        self,
        embeddingSize,
        numberofPoints,
        numberofVars,
        numberofYs,
        method="GPT",
        varibleEmbedding="NOT_VAR",
        **kwargs,
    ):
        self.embeddingSize = embeddingSize
        self.numberofPoints = numberofPoints  # number of points
        self.numberofVars = numberofVars  # input dimension (Xs)
        self.numberofYs = numberofYs  # output dimension (Ys)
        self.method = method
        self.varibleEmbedding = varibleEmbedding

        for k, v in kwargs.items():
            setattr(self, k, v)


class tNet(nn.Module):
    """
    The PointNet structure in the orginal PointNet paper:
    PointNet: Deep Learning on Point Sets for 3D Classification and Segmentation by Qi et. al. 2017
    """

    def __init__(self, config):
        super(tNet, self).__init__()

        self.activation_func = F.relu
        self.num_units = config.embeddingSize

        self.conv1 = nn.Conv1d(
            config.numberofVars + config.numberofYs, self.num_units, 1
        )
        self.conv2 = nn.Conv1d(self.num_units, 2 * self.num_units, 1)
        self.conv3 = nn.Conv1d(2 * self.num_units, 4 * self.num_units, 1)
        self.fc1 = nn.Linear(4 * self.num_units, 2 * self.num_units)
        self.fc2 = nn.Linear(2 * self.num_units, self.num_units)

        # self.relu = nn.ReLU()

        self.input_batch_norm = nn.BatchNorm1d(config.numberofVars + config.numberofYs)
        # self.input_layer_norm = nn.LayerNorm(config.numberofPoints)

        self.bn1 = nn.BatchNorm1d(self.num_units)
        self.bn2 = nn.BatchNorm1d(2 * self.num_units)
        self.bn3 = nn.BatchNorm1d(4 * self.num_units)
        self.bn4 = nn.BatchNorm1d(2 * self.num_units)
        self.bn5 = nn.BatchNorm1d(self.num_units)

    def forward(self, x):
        """
        :param x: [batch, #features, #points]
        :return:
            logit: [batch, embedding_size]
        """
        x = self.input_batch_norm(x)
        x = self.activation_func(self.bn1(self.conv1(x)))
        x = self.activation_func(self.bn2(self.conv2(x)))
        x = self.activation_func(self.bn3(self.conv3(x)))
        x, _ = torch.max(x, dim=2)  # global max pooling
        assert x.size(1) == 4 * self.num_units

        x = self.activation_func(self.bn4(self.fc1(x)))
        x = self.activation_func(self.bn5(self.fc2(x)))
        # x = self.fc2(x)

        return x


class NoisePredictionTransformer(nn.Module):
    def __init__(self, n_embd, max_seq_len, n_layer=6, n_head=8, max_timesteps=1000):
        super().__init__()
        self.pos_emb = nn.Parameter(torch.zeros(1, max_seq_len, n_embd))
        self.time_emb = nn.Embedding(max_timesteps, n_embd)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=n_embd,
            nhead=n_head,
            dim_feedforward=n_embd * 4,
            activation="gelu",
            batch_first=True,
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layer)

    def forward(self, x_t, t, condition):
        B, L, _ = x_t.shape
        pos_emb = self.pos_emb[:, :L, :]  # [1, L, n_embd]
        time_emb = self.time_emb(t)
        if time_emb.dim() == 1:  # Scalar t case, [512]
            time_emb = time_emb.unsqueeze(0)  # [1, 512]
        time_emb = time_emb.unsqueeze(1)  # [1, 1, n_embd]
        condition = condition.unsqueeze(1)  # [B, 1, n_embd]

        # Expand to match sequence length L
        time_emb = time_emb.expand(B, L, -1)  # [B, L, n_embd]

        x = x_t + pos_emb + time_emb + condition
        return self.encoder(x)


# Symbolic Diffusion with Hybrid Loss
class SymbolicGaussianDiffusion(nn.Module):
    def __init__(
        self,
        tnet_config,
        vocab_size,
        max_seq_len,
        padding_idx: int = 0,
        max_num_vars: int = 9,
        n_layer=6,
        n_head=8,
        n_embd=512,
        timesteps=1000,
        beta_start=0.0001,
        beta_end=0.02,
        ce_weight=1.0,  # Weight for CE loss relative to MSE
    ):
        super().__init__()
        self.vocab_size = vocab_size
        self.max_seq_len = max_seq_len
        self.padding_idx = padding_idx
        self.n_embd = n_embd
        self.timesteps = timesteps
        self.ce_weight = ce_weight

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Embedding layers
        self.tok_emb = nn.Embedding(vocab_size, n_embd, padding_idx=self.padding_idx)
        self.vars_emb = nn.Embedding(max_num_vars, n_embd)

        # Decoder
        self.decoder = nn.Linear(n_embd, vocab_size, bias=False)
        self.decoder.weight = self.tok_emb.weight

        # Models
        self.tnet = tNet(tnet_config)
        self.model = NoisePredictionTransformer(
            n_embd, max_seq_len, n_layer, n_head, timesteps
        )

        # Noise schedule
        self.register_buffer("beta", torch.linspace(beta_start, beta_end, timesteps))
        self.register_buffer("alpha", 1.0 - self.beta)
        self.register_buffer("alpha_bar", torch.cumprod(self.alpha, dim=0))

    def q_sample(self, x_start, t, noise=None):
        noise = torch.randn_like(x_start)
        sqrt_alpha_bar = torch.sqrt(self.alpha_bar[t]).view(-1, 1, 1)
        sqrt_one_minus_alpha_bar = torch.sqrt(1 - self.alpha_bar[t]).view(-1, 1, 1)

        x_t = sqrt_alpha_bar * x_start + sqrt_one_minus_alpha_bar * noise
        return x_t

    def p_mean_variance(self, x, t, t_next, condition):
        alpha_t = self.alpha[t]
        alpha_bar_t = self.alpha_bar[t]
        alpha_bar_t_next = self.alpha_bar[t_next]
        beta_t = self.beta[t]

        x_start_pred = self.model(x, t.long(), condition)

        coeff1 = torch.sqrt(alpha_bar_t_next) * beta_t / (1 - alpha_bar_t)
        coeff2 = torch.sqrt(alpha_t) * (1 - alpha_bar_t_next) / (1 - alpha_bar_t)
        mean = coeff1 * x_start_pred + coeff2 * x
        variance = (1 - alpha_bar_t_next) / (1 - alpha_bar_t) * beta_t
        return mean, variance

    @torch.no_grad()
    def p_sample(self, x, t, t_next, condition):
        mean, variance = self.p_mean_variance(x, t, t_next, condition)
        if torch.all(t_next == 0):
            return mean
        noise = torch.randn_like(x)
        return mean + torch.sqrt(variance) * noise

    @torch.no_grad()
    def sample(self, points, variables, batch_size=16):
        condition = self.tnet(points) + self.vars_emb(variables)
        shape = (batch_size, self.max_seq_len, self.n_embd)
        x = torch.randn(shape, device=self.device)
        steps = torch.arange(
            self.timesteps - 1, -1, -1, device=self.device
        )  # Fix: start at timesteps-1

        for i in tqdm(
            range(self.timesteps), desc="sampling loop", total=self.timesteps
        ):
            t = steps[i]
            t_next = (
                steps[i + 1]
                if i + 1 < self.timesteps
                else torch.tensor(0, device=self.device)
            )
            x = self.p_sample(x, t, t_next, condition)

        # Map embeddings to token indices via decoder
        logits = self.decoder(x)  # [B, L, vocab_size]
        token_indices = torch.argmax(logits, dim=-1)  # [B, L]
        return token_indices

    def p_losses(self, x_start, points, tokens, variables, t, noise=None):
        """Hybrid loss: MSE on embeddings + CE on tokens."""
        noise = torch.randn_like(x_start)
        x_t = self.q_sample(x_start, t, noise)
        condition = self.tnet(points) + self.vars_emb(variables)
        x_start_pred = self.model(x_t, t.long(), condition)

        # MSE loss on embeddings
        mse_loss = torch.tensor(0.0, device=self.device)

        # CE loss on tokens
        logits = self.decoder(x_start_pred)  # [B, L, vocab_size]
        ce_loss = (
            F.cross_entropy(
                logits.view(-1, self.vocab_size),  # [B*L, vocab_size]
                tokens.view(-1),  # [B*L]
                ignore_index=self.padding_idx,  # Assuming padding_idx=0
                reduction="none",
            )
            .view(tokens.shape)
            .mean()
        )  # [B]

        # Combine losses
        total_loss = mse_loss + self.ce_weight * ce_loss
        return total_loss, mse_loss, ce_loss

    def forward(self, points, tokens, t):
        token_emb = self.tok_emb(tokens)
        total_loss, mse_loss, ce_loss = self.p_losses(token_emb, tokens, points, t)
        return total_loss, mse_loss, ce_loss


In [33]:
import torch
import json
from scipy.optimize import minimize
from math import log

@torch.no_grad()
def test_model(model, test_loader, textTest, train_dataset, device):
    results = {'target': [], 'predicted_skeleton' : [], 'predicted': [], 'error': []}

    for batch_idx, (_, tokens, points, variables) in enumerate(test_loader):
        points = points.to(device)    # [B, 2, 250]
        tokens = tokens.to(device)    # [B, L]
        variables = variables.to(device)  # [B]

        B = points.shape[0]

        generated_tokens = model.sample(points, variables, B)

        for i in range(batch_size):
            # Ground truth
            eq = ''.join([train_dataset.itos[int(i)] for i in tokens[0]])
            eq = eq.strip(train_dataset.paddingToken).split('>')
            eq = eq[0] #if len(eq[0])>=1 else eq[1]
            eq = eq.strip('<').strip(">")

            # Predicted
            predicted_tokens = generated_tokens[i].cpu().numpy()
            predicted = ''.join([train_dataset.itos[int(idx)] for idx in predicted_tokens])
            predicted = predicted.strip(train_dataset.paddingToken).split('>')
            predicted = predicted[0] if len(predicted[0]) >= 1 else predicted[1]
            predicted = predicted.strip('<').strip(">")
            predicted = predicted.replace('Ce', 'C*e')
            predicted_skeleton = predicted


            t = json.loads(textTest[i])
            # train a regressor to find the constants (too slow)
            c = [1.0 for i,x in enumerate(predicted) if x=='C'] # initialize coefficients as 1
            # c[-1] = 0 # initialize the constant as zero
            b = [(-2,2) for i,x in enumerate(predicted) if x=='C']  # bounds on variables
            try:
                if len(c) != 0:
                    # This is the bottleneck in our algorithm
                    # for easier comparison, we are using minimize package
                    cHat = minimize(lossFunc, c, #bounds=b,
                                args=(predicted, t['X'], t['Y']))

                    predicted = predicted.replace('C','{}').format(*cHat.x)
            except ValueError:
                raise 'Err: Wrong Equation {}'.format(predicted)
            except Exception as e:
                raise 'Err: Wrong Equation {}, Err: {}'.format(predicted, e)

            Ys = [] #t['YT']
            Yhats = []
            for xs in t['XT']:
              try:
                  eqTmp = eq + ''  # copy eq
                  eqTmp = eqTmp.replace(' ', '')
                  eqTmp = eqTmp.replace('\n', '')
                  for i, x in enumerate(xs):
                      # replace xi with the value in the eq
                      eqTmp = eqTmp.replace('x{}'.format(i + 1), str(x))
                      if ',' in eqTmp:
                          assert 'There is a "," in the equation!'
                  YEval = eval(eqTmp)
              except Exception as e:
                  continue
                  print(f"TA: Invalid equation encountered. Eq: {eqTmp}, Reason: {e}")
                  YEval = 100  # Assign a default value or handle as needed
                  #raise
                  YEval = 100
              Ys.append(YEval)

              try:
                  eqTmp = predicted + ''  # copy eq
                  eqTmp = eqTmp.replace(' ', '')
                  eqTmp = eqTmp.replace('\n', '')
                  for i, x in enumerate(xs):
                      # replace xi with the value in the eq
                      eqTmp = eqTmp.replace('x{}'.format(i + 1), str(x))
                      if ',' in eqTmp:
                          assert 'There is a "," in the equation!'
                  Yhat = eval(eqTmp)
              except Exception as e:
                  continue
                  print(f"PR: Invalid equation encountered. Eq: {eqTmp}, Reason: {e}")
                  Yhat = 100  # Assign a default value or handle as needed
              Yhats.append(Yhat)
            err = relativeErr(Ys,Yhats, info=True)


            results['target'].append(eq)
            results['predicted_skeleton'].append(predicted_skeleton)
            results['predicted'].append(predicted)
            results['error'].append(err)

            print(f"\nSample {batch_idx * batch_size + i + 1}:")
            print(f"Target: {eq}")
            print(f"Predicted Skeleton: {predicted_skeleton}")
            print(f"Predicted: {predicted}")
            print(f"Relative Error: {err:.6f}")
            print("-" * 50)

    return results

In [43]:
pconfig = PointNetConfig(
    embeddingSize=n_embd,
    numberofPoints=250,
    numberofVars=1,
    numberofYs=1,
)
model = SymbolicGaussianDiffusion(
    tnet_config=pconfig,
    vocab_size=train_dataset.vocab_size,
    max_seq_len=32,
    padding_idx=test_dataset.paddingID,
    max_num_vars=9,
    n_layer=4,
    n_head=4,
    n_embd=n_embd,
    timesteps=timesteps,
    beta_start=0.0001,
    beta_end=0.02,
).to(device)


print(train_dataset.itos)

model_path = "/content/drive/MyDrive/Colab Notebooks/STAT946_proj/models/best_model_115.pth"
model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
model.eval()

print("Testing SymbolicDiffusion model...")
test_results = test_model(model, test_loader, textTest, train_dataset, device)

print("\nSummary:")
for i in range(len(test_results['target'])):
    print(f"Sample {i+1}:")
    print(f"  Target: {test_results['target'][i]}")
    print(f"  Predicted Skeleton: {test_results['predicted_skeleton'][i]}")
    print(f"  Predicted: {test_results['predicted'][i]}")
    print(f"  Error: {test_results['error'][i]:.6f}")

{0: '(', 1: ')', 2: '*', 3: '**', 4: '+', 5: '/', 6: '0', 7: '1', 8: '2', 9: '3', 10: '4', 11: '5', 12: '6', 13: '7', 14: '8', 15: '9', 16: ':', 17: '<', 18: '>', 19: 'C', 20: 'T', 21: '_', 22: 'cos', 23: 'exp', 24: 'log', 25: 'sin', 26: 'x1'}
Testing SymbolicDiffusion model...


sampling loop: 100%|██████████| 1000/1000 [00:02<00:00, 427.29it/s]



Sample 1:
Target: C*exp(C*x1/(C*x1))+C
Predicted Skeleton: C*x1+C
Predicted: -8.892626690570893e-09*x1+-1.1886541304097147
Relative Error: 100.000000
--------------------------------------------------


sampling loop: 100%|██████████| 1000/1000 [00:02<00:00, 489.75it/s]



Sample 2:
Target: C*exp(C*x1)*C*cos(C*x1**2+C*x1)+C
Predicted Skeleton: C*x1+C
Predicted: -8.892626690570893e-09*x1+-1.1886541304097147
Relative Error: 100.000000
--------------------------------------------------


...


Sample 182:
Target: C*exp(C*x1)+C
Predicted Skeleton: C*x1+C
Predicted: -8.892626690570893e-09*x1+-1.1886541304097147
Relative Error: 100.000000
--------------------------------------------------


sampling loop: 100%|██████████| 1000/1000 [00:01<00:00, 500.86it/s]



Sample 183:
Target: C*log(C*x1+C)+C*cos(C*x1)+C
Predicted Skeleton: C*sin(C*x1)6C
Predicted: 1.0*sin(1.0*x1)61.0
Relative Error: 100.000000
--------------------------------------------------


sampling loop:  11%|█         | 107/1000 [00:00<00:02, 440.34it/s]


KeyboardInterrupt: 